In [1]:
from astropy.table import Table, join, vstack
import astropy.units as asu
import numpy as np

In [2]:
import re

In [2]:
ref = 'Osmond_2004'

In [3]:
temp = Table.read('Lx/{}/tab_4.tex'.format(ref))

In [4]:
temp = temp[temp['\TX'] != 'n/a']

In [5]:
good = temp['Group', '\TX', '\Z', 'log \LX', 'log \LXrfh']

In [6]:
good.rename_columns(['\TX', '\Z', 'log \LX', 'log \LXrfh'], ['Tx', 'Z', 'LogLxcut', 'LogLx500'])

In [7]:
# units = Table(good[0]).copy()
good.remove_row(0)

In [8]:
res = {key: [] for key in ['Tx', 'Z', 'LogLxcut', 'LogLx500']}
for key in ['Tx', 'Z', 'LogLxcut', 'LogLx500']:
    res[key + 'err'] = []
res['Group'] = []
for i in range(len(good)):
    res['Group'].append(good['Group'][i])
    for key in ['Tx', 'Z', 'LogLxcut', 'LogLx500']:
        valstr = good[key][i].split()
        res[key].append(eval(valstr[0]))
        res[key + 'err'].append(eval(valstr[-1]))
res = Table(res)

for key in ['LogLxcut', 'LogLx500']:
    for app in ['', 'err']:
        res[key + app].unit = asu.erg/asu.s
for key in ['Tx']:
    for app in ['', 'err']:
        res[key + app].unit = asu.keV
for key in ['Z']:
    for app in ['', 'err']:
        res[key + app].unit = 'Zsun'

In [9]:
res.write('Lx/Data/{}.ecsv'.format(ref))

In [4]:
ref = 'Pratt_2009'

In [5]:
def extract_errs(instr):
    main = instr.replace('$', '')
    ovals = re.split('_|\^|{|}|\$', main)
    vals = []
    for val in ovals:
        if val != '':
            vals.append(val)
#     print(main, vals, ovals)
    return eval(vals[0]), eval(vals[1]), eval(vals[2])

In [16]:
temp = Table.read('Lx/{}/TabB1.tex'.format(ref))

In [17]:
temp.remove_row(0)

In [18]:
temp.rename_columns(['\\multicolumn{1}{c}{Cluster}',
                     '\\multicolumn{1}{c}{$z$}',
                     '\\multicolumn{1}{c}{$T_1$}',
                     '\\multicolumn{1}{c}{$L\\, {[0.1-2.4]}_1$}',
                     '\\multicolumn{1}{c}{$L\\, {[0.5-2]}_1$}',
                     '\\multicolumn{1}{c}{$T_2$}',
                     '\\multicolumn{1}{c}{$L\\, {[0.1-2.4]}_2$}',
                     '\\multicolumn{1}{c}{$L\\, {[0.5-2]}_2$}'],
                    ['Cluster', 'z', 'T1', 'L_01-24_1', 'L_05-2_1', 'T2', 'L_01-24_2',
                     'L_05-2_2'])

In [21]:
keys = ['T1', 'L_01-24_1', 'L_05-2_1', 'T2', 'L_01-24_2', 'L_05-2_2']

res = {'Cluster': temp['Cluster'].data, 'z': temp['z'].data.astype(float)}
for key in keys:
    res[key] = []
    res[key + '_lerr'] = []
    res[key + '_uerr'] = []

for it in temp:
    for key in keys:
        tstr = it[key]
        tmean, tlerr, tuerr = extract_errs(tstr)
        res[key].append(tmean)
        res[key + '_lerr'].append(tlerr)
        res[key + '_uerr'].append(tuerr)
        
for key in ['T1', 'T2']:
    for app in ['', '_lerr', '_uerr']:
        res[key + app] = np.array(res[key + app]) * asu.keV
for key in ['L_01-24_1', 'L_05-2_1', 'L_01-24_2', 'L_05-2_2']:
    for app in ['', '_lerr', '_uerr']:
        unit = (1e44 * asu.erg/asu.s).to('1e44 erg/s')
        res[key + app] = np.array(res[key + app]) * unit
#     res[key] = np.array(res[key])
res = Table(res)

In [24]:
res.write('Lx/Data/{}.ecsv'.format(ref), overwrite=True)

In [5]:
ref = 'Eckmiller_2011'

In [6]:
temp1 = Table.read('Lx/{}/tab1.tex'.format(ref))

temp2 = Table.read('Lx/{}/tab3-1.tex'.format(ref))

temp3 = Table.read('Lx/{}/tab3-2.tex'.format(ref))

temp4 = Table.read('Lx/{}/tab2.tex'.format(ref))

temp1.remove_row(0)

temp2.remove_row(0)

temp3.remove_row(0)

temp4.remove_row(0)

temp4.rename_column('Name', 'Group Name')

In [7]:
temp = join(temp2, temp3, keys=['Group Name'])

In [8]:
temp = join(temp, temp4, keys=['Group Name'])

In [9]:
temp = join(temp1, temp, keys=['Group Name'])

In [10]:
temp.rename_columns(['$z$', '$L_{\\text{x}}$', '$L_{\\text{x}}$ error', '$k_{\\text{B}}~T$',
                     '$r_{2500}$', '$M_{2500}$', '$r_{500}$', '$M_{500}$', '$M_{\\text{g,2500}}$',
                     '$M_{\\text{g,500}}$', '$f_{\\text{g,2500}}$', '$f_{\\text{g,500}}$', '$Z$'],
                     ['z', 'Lx', 'Lxerr', 'T', 'R2500', 'M2500', 'R500', 'M500', 'Mg2500',
                      'Mg500', 'fg2500', 'fg500', 'Z'])

In [11]:
good = temp['Group Name', 'z', 'Lx', 'Lxerr', 'T', 'R2500', 'M2500', 'R500',
            'M500', 'Mg2500', 'Mg500', 'fg2500', 'fg500', 'Z', 'Core']

In [12]:
keys = ['T', 'R2500', 'M2500', 'R500',
            'M500', 'Mg2500', 'Mg500', 'fg2500', 'fg500', 'Z']

In [21]:
def rm_em(tlist):
    res = []
    for it in tlist:
        if len(it) != 0:
            res.append(it)
    return res

In [22]:
res = {}

for key in ['Group Name', 'z', 'Lx', 'Lxerr', 'Core']:
    res[key] = good[key].data
    
for key in keys:
    for app in ['', 'err']:
        res[key + app] = []

for key in keys:
#     for app in ['', 'err']:
    for i in range(len(good)):
        tempstr = re.split('\$|\\\\pm', good[key][i])
        tempvals = rm_em(tempstr)
        tmean, terr = eval(tempvals[0]), eval(tempvals[1])
        res[key].append(tmean)
        res[key + 'err'].append(terr)

res = Table(res)

for key in ['M']:
    for app1 in ['500', '2500']:
        for app2 in ['', 'err']:
            unit = 1e13 * asu.Msun/asu.littleh
            res[key + app1 + app2].unit = unit
            
for key in ['M']:
    for app1 in ['g500', 'g2500']:
        for app2 in ['', 'err']:
            unit = 1e13 * asu.Msun/asu.littleh**(5/2)
            res[key + app1 + app2].unit = unit

for key in ['R']:
    for app1 in ['500', '2500']:
        for app2 in ['', 'err']:
            unit = asu.Mpc/asu.littleh
            res[key + app1 + app2].unit = unit     
            
for key in ['T']:
    for app in ['', 'err']:
        res[key + app].unit = asu.keV
        
for key in ['f']:
    for app1 in ['g500', 'g2500']:
        for app2 in ['', 'err']:
            unit = asu.littleh**(-3/2)
            res[key + app1 + app2].unit = unit

res['Lx'].unit = 1e44 * asu.erg/asu.s/asu.littleh**2
res['Lxerr'].unit = asu.percent
for key in ['Z']:
    for app in ['', 'err']:
        res[key + app].unit = 'Zsun'

In [23]:
res.write('Lx/Data/{}.ecsv'.format(ref), overwrite=True)

In [215]:
ref = 'Leauthaud_2010' # Ignore this dataset for now since its relatively higher z

In [216]:
# temp = Table.read('{}/tab1.tex'.format(ref))

In [217]:
# temp.rename_columns(['$\\langle {L_{X}.E(z)^{-1}} \\rangle$', '$f_{\\rm CC}$\\tablenotemark{a}',
#                      '$\\langle {M}_{200} \\rangle$', '$\\langle z \\rangle$'])

In [290]:
ref = 'Sun_2009'

In [33]:
def splitsun(tstr):
    if 'pm' in tstr:
        slist = re.findall(r"[-+]?[.]?[\d]+(?:,\d\d\d)*[\.]?\d*(?:[eE][-+]?\d+)?", tstr)
        if len(slist) != 2:
            raise Exception("error")
        tmean, terr = eval(slist[0]), eval(slist[1])
        tlerr, tuerr = -terr, terr
    else:
        slist = re.findall(r"[-+]?[.]?[\d]+(?:,\d\d\d)*[\.]?\d*(?:[eE][-+]?\d+)?", tstr)
        if len(slist) == 1:
            tmean, tlerr, tuerr = eval(slist[0]), np.nan, np.nan
        elif len(slist) != 3:
            raise Exception(slist, "error")
        else:
            tmean, terr = eval(slist[0]), (eval(slist[1]), eval(slist[2]))
            tlerr, tuerr = min(terr), max(terr)
    if tlerr * tuerr > 0:
        raise Exception(tlerr, tuerr, 'Wrong lower/upper error!')
    return tmean, tlerr, tuerr

In [291]:
temp1 = Table.read('Lx/{}/tab3.tex'.format(ref))
temp1.remove_row(0)

In [292]:
temp2 = Table.read('Lx/{}/tab4.tex'.format(ref))

In [293]:
temp = join(temp1, temp2, keys='Group')

In [295]:
temp.rename_columns(['$T_{500}$\\tablenotemark{a}', '$T_{2500}$', '$r_{500}$\\tablenotemark{a}',
                     '$r_{2500}$', '$M_{500}$\\tablenotemark{a}',
                     '$f_{\\rm gas, 500}$\\tablenotemark{a}', '$f_{\\rm gas, 2500}$',
                     '$K_{500}$\\tablenotemark{a}', '$K_{2500}$'],
                    ['T500', 'T2500', 'R500', 'R2500', 'M500', 'fg500', 'fg2500', 'K500',
                     'K2500'])

In [296]:
good = temp['Group', 'T500', 'T2500', 'R500', 'R2500', 'M500', 'fg500', 'fg2500', 'K500',
                     'K2500']

In [297]:
keys = ['T500', 'T2500', 'R500', 'R2500', 'M500', 'fg500', 'fg2500', 'K500', 'K2500']

res = {'Group': good['Group'].data}

for key in keys:
    for app in ['', '_lerr', '_uerr']:
        res[key + app] = []
        
for i in range(len(good)):
    for key in keys:
        if str(good[key][i]) == '--':
            tmean, tlerr, tuerr = np.nan, np.nan, np.nan
        else:
            tmean, tlerr, tuerr = splitsun(good[key][i])
        res[key].append(tmean)
        res[key + '_lerr'].append(tlerr)
        res[key + '_uerr'].append(tuerr)
res = Table(res)

for key in ['T']:
    for app1 in ['500', '2500']:
        for app2 in ['', '_lerr', '_uerr']:
            res[key + app1 + app2].unit = asu.keV
for key in ['R']:
    for app1 in ['500', '2500']:
        for app2 in ['', '_lerr', '_uerr']:
            res[key + app1 + app2].unit = asu.kpc
for key in ['M']:
    for app1 in ['500']:
        for app2 in ['', '_lerr', '_uerr']:
            res[key + app1 + app2].unit = 1e13 * asu.Msun
for key in ['K']:
    for app1 in ['500', '2500']:
        for app2 in ['', '_lerr', '_uerr']:
            res[key + app1 + app2].unit = asu.keV * asu.cm**2

In [298]:
res.write('Lx/Data/{}.ecsv'.format(ref))

In [58]:
ref = 'Chiu_2018'

In [59]:
def splitchiu(tstr):
    if 'pm' in tstr:
        slist = re.findall(r"[-+]?[.]?[\d]+(?:,\d\d\d)*[\.]?\d*(?:[eE][-+]?\d+)?", tstr)
        if len(slist) != 2:
            raise Exception("error")
        tmean, terr = eval(slist[0]), eval(slist[1])
    else:
        slist = re.findall(r"[-+]?[.]?[\d]+(?:,\d\d\d)*[\.]?\d*(?:[eE][-+]?\d+)?", tstr)
        if len(slist) == 1:
            tmean, terr = eval(slist[0]), np.nan
        elif len(slist) == 0:
            tmean, terr = np.nan, np.nan
        else:
            raise Exception(slist, 'error')
    return tmean, terr

In [60]:
temp1 = Table.read('Mass/Chiu_2018/tab1.tex')

In [61]:
temp2 = Table.read('Mass/Chiu_2018/tab2.tex')
temp2.rename_columns(['col0', '_1'], ['Name', 'fc'])
temp2.remove_row(0)

In [62]:
temp = join(temp1, temp2, keys='Name')

In [63]:
len(temp), len(temp1), len(temp2)

(91, 91, 91)

In [64]:
temp.rename_columns(['\\redshift\\', '\\Rfiveoo', '$\\Mfiveoo$', '$\\Mgas$', '$\\Mstar$', '$\\Mbary$'],
                    ['z', 'R500', 'M500', 'Mgas', 'Mstar', 'Mbar'])

In [65]:
good = temp['Name', 'z', 'R500', 'M500', 'Mgas', 'Mstar', 'Mbar', 'fc']

In [66]:
keys = ['z', 'R500', 'M500', 'Mgas', 'Mstar', 'Mbar', 'fc']

res = {'Name': good['Name'].data}
for key in keys:
    for app in ['', 'err']:
        res[key + app] = []
        
for i in range(len(good)):
    for key in keys:
#         print(good[i][key])
        tstr = good[key][i]
        tmean, terr = splitchiu(tstr)
        res[key].append(tmean)
        res[key + 'err'].append(terr)
res = Table(res)

res.remove_columns(['zerr', 'R500err'])

for key in ['M500']:
    for app in ['', 'err']:
        res[key + app].unit = 1e14 * asu.Msun
        
for key in ['Mgas', 'Mbar']:
    for app in ['', 'err']:
        res[key + app].unit = 1e13 * asu.Msun
        
for key in ['Mstar']:
    for app in ['', 'err']:
        res[key + app].unit = 1e12 * asu.Msun
        
res['R500'].unit = asu.arcmin

In [67]:
res.write('Mass/Data/{}.ecsv'.format(ref))

In [68]:
ref = 'Gonzalez_2013'

In [98]:
def splitgon(tstr):
    if 'pm' in tstr:
        slist = re.findall(r"[-+]?[.]?[\d]+(?:,\d\d\d)*[\.]?\d*(?:[eE][-+]?\d+)?", tstr)
        if len(slist) != 2:
            raise Exception("error")
        tmean, terr = eval(slist[0]), eval(slist[1])
    else:
        slist = re.findall(r"[-+]?[.]?[\d]+(?:,\d\d\d)*[\.]?\d*(?:[eE][-+]?\d+)?", tstr)
        if len(slist) == 1:
            tmean, terr = eval(slist[0]), np.nan
        elif len(slist) == 0:
            tmean, terr = np.nan, np.nan
        else:
            raise Exception(slist, 'error')
    return tmean, terr

In [113]:
temp1 = Table.read('Mass/Gonzalez_2013/tab2.tex')
temp1.rename_column('\\colhead{Cluster}', 'Cluster')
temp1.remove_row(0)

temp2 = Table.read('Mass/Gonzalez_2013/tab3.tex')
temp2.rename_column('\\colhead{Cluster}', 'Cluster')
temp2.remove_row(0)

In [114]:
temp = join(temp1, temp2, keys='Cluster')

In [115]:
temp.rename_columns(['\\colhead{$z$}', '$T_{X,2}$', '\\colhead {\\rfive}', '\\colhead {\\mfive}',
                     '\\colhead {$M_{gas,500}$}', '\\colhead {$M_{\\star,2D,500}$}',
                     '\\colhead{$M_{\\star,3D,500}$}', '\\colhead{$f_{gas}$}',
                     '\\colhead{$f_{stellar}$}', '\\colhead{$f_{baryons}$}'],
                    ['z', 'Tx2', 'R500', 'M500', 'Mgas500', 'Mstar2d500', 'Mstar3d500', 'fgas',
                     'fstar', 'fbar'])

In [116]:
good = temp['Cluster', 'z', 'Tx2', 'R500', 'M500', 'Mgas500', 'Mstar2d500',
            'Mstar3d500', 'fgas', 'fstar', 'fbar']

In [119]:
keys = ['Tx2', 'R500', 'M500', 'Mgas500', 'Mstar2d500',
        'Mstar3d500', 'fgas', 'fstar', 'fbar']

res = {'Cluster': good['Cluster'].data,
       'z': good['z'].data.astype(float)}
for key in keys:
    for app in ['', 'err']:
        res[key + app] = []
        
for i in range(len(good)):
    for key in keys:
        tstr = good[key][i]
        tmean, terr = splitgon(tstr)
        res[key].append(tmean)
        res[key + app].append(terr)
res = Table(res)

for key in ['M500']:
    for app in ['', 'err']:
        res[key + app].unit = 1e14 * asu.Msun

for key in ['Mgas500', 'Mstar2d500', 'Mstar3d500']:
    for app in ['', 'err']:
        res[key + app].unit = 1e13 * asu.Msun

for key in ['R500']:
    for app in ['', 'err']:
        res[key + app].unit = asu.Mpc


In [121]:
res.write('Mass/Data/{}.ecsv'.format(ref))

In [122]:
ref = 'Lagana_2013'

In [155]:
def splitlagana(tstr):
    if 'pm' in tstr:
        slist = re.findall(r"[-+]?[.]?[\d]+(?:,\d\d\d)*[\.]?\d*(?:[eE][-+]?\d+)?", tstr)
        if len(slist) != 2:
            raise Exception("error")
        tmean, terr = eval(slist[0]), eval(slist[1])
    else:
        slist = re.findall(r"[-+]?[.]?[\d]+(?:,\d\d\d)*[\.]?\d*(?:[eE][-+]?\d+)?", tstr)
        if len(slist) == 1:
            tmean, terr = eval(slist[0]), np.nan
        elif len(slist) == 0:
            tmean, terr = np.nan, np.nan
        else:
            raise Exception(slist, 'error')
    return tmean, terr

In [181]:
temp1 = Table.read('Mass/Lagana_2013/tab1_group.tex')
temp2 = Table.read('Mass/Lagana_2013/tab1_cluster.tex')

In [182]:
temp1.rename_columns(['$r_{2500}$', '$r_{500}$', '$ <kT> $', '$M_{\\rm gas,2500} $',
                       '$M_{\\rm gas,500} $', '$M_{\\rm tot,2500} $', '$M_{\\rm tot,500} $',
                       '$M_{\\rm star,2500} $', '$M_{\\rm star,500} $'],
                      ['R2500', 'R500', 'T', 'Mgas2500', 'Mgas500', 'Mtot2500', 'Mtot500',
                       'Mstar2500', 'Mstar500'])

temp2.rename_columns(['$r_{2500}$', '$r_{500}$', '$ <kT> $', '$M_{\\rm gas,2500} $',
                       '$M_{\\rm gas,500} $', '$M_{\\rm tot,2500} $', '$M_{\\rm tot,500} $',
                       '$M_{\\rm star,2500} $', '$M_{\\rm star,500} $'],
                      ['R2500', 'R500', 'T', 'Mgas2500', 'Mgas500', 'Mtot2500', 'Mtot500',
                       'Mstar2500', 'Mstar500'])

In [183]:
good1 = temp1['Name', 'z', 'R2500', 'R500', 'T', 'Mgas2500', 'Mgas500', 'Mtot2500', 'Mtot500',
                       'Mstar2500', 'Mstar500'][1:].copy()

good2 = temp2['Name', 'z', 'R2500', 'R500', 'T', 'Mgas2500', 'Mgas500', 'Mtot2500', 'Mtot500',
                       'Mstar2500', 'Mstar500'][1:].copy()

In [184]:
cats = [good1, good2]
res = [{'Name': cat['Name'].data} for cat in cats]

for i in range(2):
    for key in ['z', 'R2500', 'R500']:
        res[i][key] = cats[i][key].data.astype(float)
    for key in ['T', 'Mgas2500', 'Mgas500', 'Mtot2500', 'Mtot500', 'Mstar2500', 'Mstar500']:
        for app in ['', 'err']:
            res[i][key + app] = []

for i in range(2):
    for k in range(len(cats[i])):
        for key in ['T', 'Mgas2500', 'Mgas500', 'Mtot2500', 'Mtot500', 'Mstar2500', 'Mstar500']:
            tstr = cats[i][key][k]
            tmean, terr = splitlagana(tstr)
            res[i][key].append(tmean)
            res[i][key + 'err'].append(terr)
    res[i] = Table(res[i])

for i in range(2):
    for app in ['', 'err']:
        res[i]['T' + app].unit = asu.keV
    for key in ['R2500', 'R500']:
        for app in ['']:
            res[i][key + app].unit = asu.kpc/asu.littleh
    for key in ['Mgas2500', 'Mgas500', 'Mstar2500', 'Mstar500']:
        for app in ['', 'err']:
            res[i][key + app].unit = 10**(12+i) * asu.Msun
    for key in ['Mtot2500', 'Mtot500']:
        for app in ['', 'err']:
            res[i][key + app].unit = 10**(13+i) * asu.Msun

In [185]:
res[0].write('Mass/Data/{}_group.ecsv'.format(ref))
res[1].write('Mass/Data/{}_cluster.ecsv'.format(ref))

In [3]:
ref = 'Lovisari_2015'

In [25]:
def splitstr(tstr):
    if 'pm' in tstr:
        slist = re.findall(r"[-+]?[.]?[\d]+(?:,\d\d\d)*[\.]?\d*(?:[eE][-+]?\d+)?", tstr)
        if len(slist) != 2:
            raise Exception("error")
        tmean, terr = eval(slist[0]), eval(slist[1])
    else:
        slist = re.findall(r"[-+]?[.]?[\d]+(?:,\d\d\d)*[\.]?\d*(?:[eE][-+]?\d+)?", tstr)
        if len(slist) == 1:
            tmean, terr = eval(slist[0]), np.nan
        elif len(slist) == 0:
            tmean, terr = np.nan, np.nan
        else:
            raise Exception(slist, 'error')
    return tmean, terr

In [7]:
temp = Table.read('Lx/{}/tab2.tex'.format(ref))

In [9]:
temp.remove_row(0)

In [12]:
temp.rename_columns(['\\rm Group \\ name',
 'kT',
 'R_{spec}',
 'R_{500}',
 'M_{500}',
 'M_{gas,500}',
 'R_{2500}',
 'M_{2500}',
 'M_{gas,2500}',
 't_{cool}',
 'L_{X,xmm}'], 
 ['Name',
 'T',
 'Rspec',
 'R500',
 'M500',
 'Mgas500',
 'R2500',
 'M2500',
 'Mgas2500',
 'tcool',
 'Lx'])

In [17]:
for i in range(len(temp)):
    temp['Name'][i] = temp['Name'][i][4:]

In [22]:
good = temp[(temp['Name'] != 'WBL154^{*}') & (temp['Name'] != 'A3390^{*}')]

In [29]:
keys = ['T', 'R500', 'M500', 'Mgas500', 'R2500', 'M2500', 'Mgas2500', 'tcool', 'Lx']

res = {'Name': good['Name'].data}
for key in keys:
    for app in ['', 'err']:
        res[key + app] = []

for i in range(len(good)):
    for key in keys:
        tstr = good[key][i]
        tmean, terr = splitstr(tstr)
        res[key].append(tmean)
        res[key + 'err'].append(terr)
res = Table(res)


for key in ['M500', 'M2500']:
    for app in ['', 'err']:
        res[key + app].unit = 1e13 * asu.Msun/asu.littleh

for key in ['Mgas500', 'Mgas2500']:
    for app in ['', 'err']:
        res[key + app].unit = 1e12 * asu.Msun/asu.littleh**(5/2)

for key in ['R500', 'R2500']:
    for app in ['', 'err']:
        res[key + app].unit = asu.kpc/asu.littleh

for key in ['T']:
    for app in ['', 'err']:
        res[key + app].unit = asu.keV

for key in ['Lx']:
    for app in ['', 'err']:
        res[key + app].unit = 1e43 * asu.erg/asu.s

for key in ['tcool']:
    for app in ['', 'err']:
        res[key + app].unit = asu.Gyr

In [31]:
res.write('Lx/Data/{}.ecsv'.format(ref))